In [1]:
from pathlib import Path
import nltk

import src.data_manager as dm
import src.text_summarization as summ
import src.topic_extraction as te

# Text summarization

In questa esercitazione vederemo come effettuare un task di single document text summarization avvalendoci dell'utilizzo di *Nasari*. Un task di text summarization prevede di definire un mapping $s: D \to D$ che mappa un documento in input $d \in D$ ad una sua versione sommaria $d^* \in D$. Le informazioni contenute in $d^*$ e mantenute rispetto a $d$ variano a seconda del contesto, obiettivi, requisiti e condizioni operative. 

In generale $|d^*| < |d|$

In questa esercitazione, il mapping $s(d) = d^*$, è definito in base alla seguente procedura di tipo **estrattivo**:

1. Estrazione del topic del titolo.
2. Estrazione del topic per ogni chunk segmentato dal corpo del testo.
3. Calcolo della rilevanza del topic espresso dal chunk rispetto a quello del documento.
4. Selezione e riordino dei chunks più rilevanti.

L'idea alla base della procedura consiste nel estrarre il **topic**l, ovvero una rappresentazione sommaria e coincisa delle informazioni espresse in una certa unità del testo. Nel nostro caso il topic è rappresentato da un'**insieme di vettori Nasari**. Una volta estratto il topic secondo una qualche strategia (in questo caso dal titolo), confrontiamo quanto le differenti "parti del testo" (chunks) siano **rilevanti rispetto** al topic uutilizzando come  misura di similarità la *Weighted Overlap*.

I chunks che andranno a costituire il documento in output $d^*$ saranno solo quelli più rilevanti rispetto al topic e che (si spera) preservino le informazioni presenti nel documento originale $d$. In questo caso parliamo di "chunks" poiché la segmentazione e analisi della rilevanza del testo può avvenire a differenti livelli: parole, frasi, paragrafi, ecc...

### Nasari

Nasari è una risorsa lessicale simile a *Babelnet*, che a differenza di quest'ultima, utilizza una rappresentazione vettoriale. I vettori presenti in Nasari reppresentano lo **span** di uno spazio multi-dimensionale in cui ogni word-form rappresenta una dimensione di tale spazio. Lo spazio è **sparso** in quanto ogni vettore un numero di componenti (lemmi) differenti.

Vediamo alcuni esempi

In [2]:
nasari = dm.Nasari(Path('data/dd-small-nasari-15.txt'))
lemmas = ['bottle', 'money', 'bank']

for lemma in lemmas:
    print("lemma: {}; vector: {}\n".format(lemma, nasari.get_vector(lemma)))

lemma: bottle; vector: [('bottle', 2225.82), ('glass', 634.59), ('wine', 380.36), ('chianti', 302.56), ('plastic', 285.24), ('water bottle', 152.6), ('pet', 139.22), ('container', 138.82), ('cork', 135.04), ('beer', 131.9), ('soft drink', 115.05), ('use', 105.64), ('privy', 92.28), ('jug', 92.08)]

lemma: money; vector: [('money', 1475.44), ('coin', 1242.3), ('currency', 1239.76), ('gold', 1074.39), ('value', 704.1), ('commodity', 596.05), ('monetary', 436.14), ('dollar', 427.6), ('silver', 385.59), ('gold standard', 363.72), ('money supply', 327.35), ('exchange', 305.08), ('central bank', 300.59), ('marx', 300.29)]

lemma: bank; vector: [('bank', 2131.9), ('banking', 479.31), ('deposit', 229.42), ('credit union', 160.08), ('money', 142.3), ('loan', 138.98), ('commercial bank', 137.9), ('central bank', 121.84), ('customer', 87.97), ('federal reserve', 77.17), ('depositor', 77.03), ('branch', 75.26), ('cheque', 74.7), ('fractional-reserve', 71.23)]



Come si può osservare ad ogni lemma è associato un vettore nasari, ovvero una lista di coppie (lemma, score), lo score viene calcolato con la *lexical specificity* dalla processo di costruzione di Nasari.


### Topic Extraction
Vediamo ora cosa significa, nella pratica, estrarre il topic del documento utilizzando una strategia *title-based*. 

Esaminiamo il documento *"Ebola-Virus-disease"*:

In [3]:
doc = dm.parse_document_sentence(Path('data/text-documents/Ebola-virus-disease.txt'))
title_extractor = te.TitleExtractor(nasari)
print("Document Title: {}\n".format(doc.title))
main_topic = title_extractor.get_topic(doc)
print("Document Topic: {}".format(main_topic))

Document Title: Ebola virus disease

Document Topic: [[('disease', 2394.01), ('health', 334.63), ('infection', 281.32), ('symptom', 200.22), ('patient', 197.26), ('cause', 190.62), ('treatment', 180.2), ('chronic', 178.71), ('pathogen', 166.77), ('kawasaki disease', 154.51), ('epidemiology', 154.45), ('ménière', 151.47), ('condition', 138.45), ('disorder', 135.0)], [('virus', 5745.67), ('viral', 1300.78), ('cell', 1172.39), ('genome', 1172.14), ('rna', 810.54), ('protein', 780.34), ('dna', 638.26), ('infect', 518.51), ('host', 462.58), ('infection', 454.04), ('capsid', 432.66), ('replication', 404.06), ('gene', 382.61), ('disease', 299.15)]]


Come si può osservare dall'output, il topic del documento è rappresentaro dai 2 vettori nasari associati ai lemmi "virus" e "disease". La parola "Ebola" presente nel titolo non ha un corrispettivo vettore nasari associato. 

Estraendo il topic anche dei chunks presenti nel corpo del testo possiamo quantificare quanto ogni chunk sia rilevante per il topic del docuemento calcolando la weighted overlap tra i vettori nasari. 

Vediamo un esempio:



In [4]:
sentence_topic = te.TopicExtractor(nasari).get_topic(doc.body[0]) # just the first chunk (here sentence)
print("Document topic: {}, Sentence topic: {}".format(len(main_topic), len(sentence_topic)))

Document topic: 2, Sentence topic: 5


Dato che i due topic sono costituiti da molteplici vettori nasari, per quantificare la rilevanza del chunk possiamo calcolare la WO media tra tutte le possibili coppie di vettori.  

In [5]:
import itertools

for v1, v2 in itertools.product(main_topic, sentence_topic):
    print(v1)
    print(v2)
    print()

[('disease', 2394.01), ('health', 334.63), ('infection', 281.32), ('symptom', 200.22), ('patient', 197.26), ('cause', 190.62), ('treatment', 180.2), ('chronic', 178.71), ('pathogen', 166.77), ('kawasaki disease', 154.51), ('epidemiology', 154.45), ('ménière', 151.47), ('condition', 138.45), ('disorder', 135.0)]
[('lemur', 2060.13), ('primate', 2038.41), ('ape', 456.84), ('monkey', 407.01), ('human', 376.41), ('tarsier', 369.35), ('chimpanzee', 299.83), ('strepsirrhines', 292.3), ('specie', 287.8), ('ring-tailed lemur', 272.83), ('prosimian', 247.4), ('madagascar', 186.63), ('mya', 173.93), ('strepsirrhini', 161.09)]

[('disease', 2394.01), ('health', 334.63), ('infection', 281.32), ('symptom', 200.22), ('patient', 197.26), ('cause', 190.62), ('treatment', 180.2), ('chronic', 178.71), ('pathogen', 166.77), ('kawasaki disease', 154.51), ('epidemiology', 154.45), ('ménière', 151.47), ('condition', 138.45), ('disorder', 135.0)]
[('virus', 5745.67), ('viral', 1300.78), ('cell', 1172.39), ('

Come si può osservare dall'output precedente, abbiamo 10 possibili coppie, quindi 10 differenti score di WO. Calcolando la media otteniamo un unico score che indica la rilevanze del chunk rispetto al topic del documento.

Ripetendo questa operazione per tutti i chunks estratti dal corpo del documento possiamo creare un ranking in base alla rilevanza e quindi selezionare solo i top-k chunks più rilevanti. La logica appena espressa è codificata nei vari metodi della classe `TextSummarizer`.

In [6]:
doc = dm.parse_document_sentence(Path('data/text-documents/Life-indoors.txt'))

summarizer = summ.TextSummarizer(main_extractor=te.TitleExtractor(nasari),
                                 chunk_extractor=te.TopicExtractor(nasari))

summary = summarizer.get_summary(doc, compression_ratio=60, debug=True)

print(summary)


The lockdown has also prompted many people to reassess their lives and what is most important to them, bringing unexpected realisations and touching moments with their families. [0.291;5]
As the world endures lockdown to slow the spread of Covid-19, people from Hong Kong to Italy are trying to find a way to continue their normal lives [0.250;1]
Asked what is the first thing she wants when life get back to normal, she says: “A hug.” [0.250;31]
After life returns to normal, I think the first thing for me is to have a big meal in a decent restaurant. [0.167;20]
In the Venezuelan capital Caracas, 51-year-old Ana Pereira lives alone with her dog and cat. [0.167;27]
The closure of workplaces has given people time with their families they never had before. [0.166;16]
Millions of people worldwide are having to embrace life under lockdown – confined to their own four walls or neighbourhoods for weeks on end as countries battle to reduce the spread of the coronavirus. [0.138;2]
In the United Sta

Nella cella precedente i chunk vengono estratti a livello di frase. Sebbene si abbia una  **maggiore granularità** nel calcolo delle informazioni rilevanti, si può osservare come viene a mancare una certa **coerenza** nel testo prodotto. Il sommario risulta quindi una **semplice giustapposizione** di frasi.

Vediamo invece cosa accade quando cambiamo il livello di analisi passando dalla frase al paragrafo:

In [7]:
doc = dm.parse_document_paragraph(Path('data/text-documents/Life-indoors.txt'))

summarizer = summ.TextSummarizer(main_extractor=te.TitleExtractor(nasari),
                                 chunk_extractor=te.TopicExtractor(nasari))

summary = summarizer.get_summary(doc, compression_ratio=60, debug=True)

print(summary)

This new way of living poses huge challenges. Teaching, working and socialising have moved online as never before. The lockdown has also prompted many people to reassess their lives and what is most important to them, bringing unexpected realisations and touching moments with their families. [0.291;3]
As the world endures lockdown to slow the spread of Covid-19, people from Hong Kong to Italy are trying to find a way to continue their normal lives [0.250;1]
Asked what is the first thing she wants when life get back to normal, she says: “A hug.” [0.250;12]
Millions of people worldwide are having to embrace life under lockdown – confined to their own four walls or neighbourhoods for weeks on end as countries battle to reduce the spread of the coronavirus. [0.138;2]
In the Venezuelan capital Caracas, 51-year-old Ana Pereira lives alone with her dog and cat. She is sitting down in front of her computer to a virtual picnic with friends, as they can’t actually meet as they have done weekly s

Da una valutazione soggettiva, il risultato prodotto lavorando a livello di paragrafi sembra più coerente e meno fragmentato rispetto all'esperimento precedente. L'osservazione non sorprende in quanto maggiore è il livello di granularità di analisi, maggiore sarà la sofisticatezza richiesta nella fase di riordinamento e riassemblaggio dei chunks estratti.

Per il semplicissimo approccio di riassemblaggio dei chunks qui implementato, non stupisce come una segmentazione a livello di paragrafo risulti qualitativamnete migliore.

### Batch Processing

Effettuiamo ora un esperimento su 4 differenti testi e con 4 livelli di compressione differenti. I testi generati sono visibili nella cartella `output`

In [13]:
doc_paths = Path('data/text-documents/').glob('*.txt')
output_path = Path('output')
output_path.mkdir(exist_ok=True) 
compression_levels = [10, 30, 60, 90]

for doc_path in doc_paths:
    doc = dm.parse_document_paragraph(doc_path)
    out_file =  output_path / doc_path.stem 
    with out_file.open('w') as file:
        for compression in compression_levels:
            summary = summarizer.get_summary(doc, compression_ratio=compression, debug=True)
            file.write("Compression Level: {}%\n".format(compression))
            file.write(summary)
            file.write("\n_____________________________________________________________\n".format(compression))
            

## Risultati

In generale analizzando i valori di relevance ottenuti per ciascun chunks emerge un'importante dinamica. I documenti come "Andy-Warhol" e "Napoleon-wiki" presentano valori di rilevance molto bassi in quanto il titolo essenzialmente è composto da una singola *named entity* dal quale non si riescono ad estrarre vettori nasari significativi che rappresentino il topic, e che di conseguenza, portano ad un basso overlap.

Analizzando qualitativamente il contenuto dei due testi, emerge un ulteriore aspetto. I due testi in questione presentano ricorrenti menzioni ad entità (luoghi, date, eventi)ed hanno un contenuto in stile **narrattivo** piuttosto che **fattuale** come i documenti "Ebola-virus-disease" e "Life-indoors". Questo aspetto potrebbe mettere in difficoltà la procedura di summarization implementata.

In generale anche in questo task, si può apprezzare come il **coverage** della risorsa lessicale utilizzata abbia un forte impatto sul risultato finale prodotto.

### Napoleon-wiki document

Questo documento presenta una peculiarità rispetto agli altri sommari generati. Come si può osservare dall'output generato i valori di rilevanza dei paragrafi sono sempre $0$, dinamica che si riflette anche sull'ordine dei paragrafi nel sommario, uguale all'ordine originale. Questo fenomeno è imputabile principalmente alla tipologia del titolo del documento in quanto contiene solamente una word-form ("Napoleon") di una *named entity* non presente in Nasari e dunque producendo un overlap vuoto.   